<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science </h1>

## Homework 1: Data Collection, Parsing, and Quick Analyses

**Harvard University**<br/>
**Fall 2020**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Tanner<br/>
<hr style='height:2px'>

In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2020-CS109A/master/themes/static/css/cs109.css").text
HTML(styles)


## Overview 

In this homework, your goal is to learn how to acquire, parse, clean, and analyze data. Toward this goal, we will address certain question about COVID, and you will scrape data directly from a website. For the remainder of the semester, we will provide you data files directly; however, since real-world problems often require gathering information from a variety of sources, including the Internet, web scraping is a highly useful skill to have.

### Instructions
- To submit your assignment, follow the instructions given in Canvas.

### Learning Objectives
- Get started using Jupyter Notebooks, which are incredibly popular, powerful, and will be our medium of programming for the duration of CS109A and CS109B.
- Become familiar with how to access and use data from various sources (i.e., web scraping and directly from files).
- Gain experience with data exploration and simple analysis.
- Become comfortable with PANDAS as a means of storing and working with data.
- Reflect on what further analysis you may wish to do with this data. For example, given the material we've covered so far, what *more* do you wish you had the ability to do (e.g., modelling, prediction, etc). That is, think about questions you may have about the data, and try to imagine what types of tools you might need to help answer your questions.

### Notes
- Exercise **responsible scraping**. Web servers can become slow or unresponsive if they receive too many requests from the same source in a short amount of time. In your code, use a delay of 2 seconds between requests. This helps to not get blocked by the target website -- imagine how frustrating it would be to have this occur. Section 1 of this homework involves saving the scraped web pages to your local machine. Thus, after completing Section 1, you do not need to re-scrape any of the pages, unless you wish to occasionally grab the latest data. 

- Web scraping requests can take several minutes. This is another reason why you should not wait until the last minute to do this homework.
- As you run a Jupyter Notebook, it maintains a running state of memory. Thus, the order in which you run cells matters and plays a crucial role; it can be easy to make mistakes based on *when* you run different cells as you develop and test your code. **Before submitting every Jupyter Notebook homework assignment, be sure to restart your Jupyter Notebook and run the entire notebook from scratch, all at once (i.e., "Kernel -> Restart & Run All")**
- We will be working with COVID data. COVID has impacted everyone in the world, and naturally some people have been greatly more affected than others. We, the teaching staff, are sensitive to this, empathize, and understand that working with COVID data may be unsettling to some. We apologize for any discomfort this may cause. Our intent with this assignment is purely pedagogical, and we'd like to remind students that data science and machine learning can be used to provide insights that can be used for good and invoke change. Toward this goal, parts of the homework are intended to shed light on the unfortunate, widespread inequality that exists. So, while this data may be unsettling, our aim is for the learned skills addressed here -- and in all future assignments -- to provide you with knowledge and confidence to do good work.

## 1. Obtaining Data (17 points)

For any given situation or scenario that we wish to understand, we will rely on having relevant data. Here, we are interested in the degree to which the SARS-CoV-2 virus has affected United States citizens (SARS-CoV-2 is the virus that causes the COVID-19 disease). The Centers for Disease Control and Prevention (CDC) provides relevant data from USAFacts.org that includes the number of confirmed COVID-19 cases on a per-county basis. Visit https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/. At the bottom of the web page, in a blue table, you should see a list of every state, each of which has its own web page.

In this exercise, we will focus on automating the downloading of each state's data (via ``Requests``). First, as we will do for every Jupyter Notebook, let's import necessary packages that we will use throughout the notebook (i.e., run the cell below). 

In [2]:
# import the necessary libraries
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

# NOTE: files will be saved to this directory, so you need to ensure
# that it exists on your system first (it should be visible from the
# directory of where you are running this Notebook file)
# i.e.,
# >> ls
# cs109a_hw1_student.ipynb
# election2016_by_county.csv
# data/
state_dir = "data/"

In [3]:
# we define this for convenience, as every state's url begins with this prefix
base_url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'

<div class='exercise'><b> Exercise 1.1 [1 pt]: Fetching Website data via Requests</b>

Fetch the web page located at `base_url` and save the request's returned object (a Response object) to a variable named `home_page`.
</div>

In [4]:
# YOUR CODE HERE
home_page = requests.get(base_url)
# END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 1.2 [2 pts]:</b> In the cell below:
    
- Write a line of code that prints to the screen the status of `home_page` (the web page's returned object). You should receive a code of 200 if the request was successful; then,

- Write code that prints the entire contents of `home_page`</div>


In [5]:
# YOUR CODE HERE
print(f"Status: {home_page.status_code}")
print(f"Contents: {home_page.text}")
# END OF YOUR CODE HERE

Status: 200
Contents: <!doctype html><html lang="en"><head><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-c11b817f31177e0b4d1","applicationID":"645680062","applicationTime":721.483669,"transactionName":"ZwZaNUEFVhZZAkNRWl5Mdg5BCVkJURtSXGBCChdL","queueTime":0,"ttGuid":"7661fdfdeceee032","agentToken":null}; (window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:false}};(window.NREUM||(NREUM={})).loader_config={xpid:"VgMCVV5VCBAGUFRUDwEHUFY=",licenseKey:"NRJS-c11b817f31177e0b4d1",applicationID:"645680062"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){

<div class='exercise'><b> Exercise 1.3 [1 pt]:</b>
    
In the cell below, create a new BeautifulSoup object that parses the `home_page` as an HTML document (can be done with 1 line of code)</div>

In [6]:
# YOUR CODE HERE
soup = BeautifulSoup(home_page.text, "html.parser")
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 1.4 [8 pts]:</b>
    
In the cell below, write code that uses the BeautifulSoup object to parse through the home page in order to extract the link for every state. Feel free to use Regular Expressions, in conjunction with any BeautifulSoup parsing. Specifically, the goal is to populate the `state_urls` dictionary by setting each key to be the state name and the value to be the full URL. When complete, there will be 51 keys (50 states + 1 for DC).

### AS A CRITICAL EXAMPLE:###
Within `state_urls`, one of your <key, value> pairs should be:

``"District of Columbia" : "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia"``

The casing here is **incredibly** important because later, in Exercise 4, you will merge your data with another dataset that has casing of this form. Thus, our key here should be `District of Columbia` and not `District Of Columbia` or `district-of-columbia`.


**NOTES:**
- There are _many_ solutions, but you may find it easiest to use Regular Expression(s)
- Pay attention to the casing example above, so that your later Exercises go smoothly. 
</div>

In [7]:
state_urls = {}
base = "https://usafacts.org"
# YOUR CODE HERE
rxp = re.compile("/state/([a-z]-*)")
for link in soup.find_all("a"):
    if re.search(rxp, link.get("href")):
        state_urls[link.get_text()] = base + link.get("href")
# END OF YOUR CODE HERE

Run the cell below to help ensure your formatting is correct and has 51 <key, value> pairs.

In [8]:
# SANITY CHECK
if len(state_urls.keys()) != 51 or \
state_urls["District of Columbia"] != "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia":
    print("** 1.4 is incorrect")
else:
    print("** 1.4 might be correct")

** 1.4 might be correct


We wish to use the data without having to re-download it every time. So, let's save each webpage to our local hard drive. **NOTE: It's probably okay to download all of the state web pages a few times a day, but it's safer to keep it to a minimum.**

<div class='exercise'><b> Exercise 1.5 [5 pts]:</b>
    
In the cell below, we will iterate through all <key, value> items in `state_urls`. Your job is to make a web request for each URL and save the **contents** out to a file on your hard drive (use `state_dir` as the prefix to the path.) 

**NOTES:**
- Leave the 2 second pause
- You should be saving to a file the actual content of the webpage, not a BeautifulSoup object. That is, you should be able to open the saved files in an editor and see the HTML code, just as you could if you were to view the webpage in your browser and click 'View Page Source'.
- See [official Python documentation](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files) for details on how to read/write files to disk
- You should have saved 51 different files to your hard drive.
</div>

In [9]:
# 1.5 (4 pts) -- save each webpage to disk
for state, url in state_urls.items():
    
    # YOUR CODE HERE
    page = requests.get(url)
    if page.status_code == 200:
        content = page.text
        f = open(state_dir + state, "a")
        f.write(content)
        f.close()
    # END OF YOUR CODE HERE
    
    sleep(2) # LEAVE THIS IN

FileNotFoundError: [Errno 2] No such file or directory: 'data/Alabama'

## 2. Loading and Exploring Data (22 pts)
Now, let's actually use the data! Fortunately, it's saved to our local machine, so we don't need to re-crawl the data every time we wish to access it. We want you to understand that PANDAS is a library of useful data structures and operations, but we also wish to remind you that it isn't magic and it isn't the _only_ way to do Data Science; it's just a tool to help, and you could do the same operations without PANDAS. Thus, here we ask you to perform a few operations without using PANDAS, and then in Exercise 3 we will use PANDAS.

**Terminology Notice:** In the United States, every state is comprised of many **counties.** You can think of a **county** as being a pretty large district. 

First, run the cell below to construct `state_info`

In [240]:
state_info = [(state, state_dir + state) for state in state_urls.keys()]

<div class='exercise'><b> Exercise 2.1 [10 pts]: Parsing and storing data</b>
    
Complete the `load_covid_data()` function, which:

- Takes as input `state_info`, which is a list of tuples: (state name, path to the corresponding file)
- Parses the contents of the file and extracts for **each county**:
    - \# of confirmed cases (total)
    - \# of deaths
    - \# of confirmed cases (per 100k)
- Stores in a **non-PANDAS** data structure named `covid_data` the above 3 pieces of data, **for every county across every state**
- Returns `county_counts`
    <font color='blue'>

**NOTES:**
- To be clear, as of September 7, 2020, the webpage for Alabama currently lists 67 counties. District of Columbia has 1 county, and Wyoming has 23. Here we are asking you to store in `covid_data` *all counties* across every state. So, later, if we were wished to access just Wyoming's information, you could easily retrieve such for each of its 23 counties, or the info for any of the 67 counties in Alabama. 
- `covid_data` **must not be a PANDAS data structure;** it must use a combination of lists and/or dictionaries. It's up to you to decide how to organize this, e.g., a lists of lists of lists, or a list of dictionaries, or a dictionary of dictionaries, or a dictionary of lists of lists, etc. A guiding decision should be ease of access for computing basic stats (Exercises 2.2, 2.3, and 2.4)
</div>

In [241]:
# parses all county-level COVID data from the
# passed-in list of (state, filepath) tuples 
def load_covid_data(state_info):

    covid_data = {}
    
    # YOUR CODE HERE
    for state, path in state_info:
        state_details = {}
        page_text = open(path, "r").read()
        soup = BeautifulSoup(page_text, "html.parser")
        for row in soup.find_all('tr'):
            if row.find('th').find('a'):
                details = {}
                a = row.find('th').find('a').text
                td = row.find_all('td')
                details['cases'] = int(td[0].text.replace(',', ''))
                details['deaths'] = int(td[1].text.replace(',', ''))
                details['cases_per_100k'] = float(td[2].text.replace(',', ''))
                state_details[a] = details
        covid_data[state] = state_details
    # END OF YOUR CODE HERE
    return covid_data

Run the cell below (no changes necessary) to execute your code above

In [242]:
covid_data = load_covid_data(state_info)

<div class='exercise'><b> Exercise 2.2 [4 pts]: Simple analytics</b>
    
Complete the `calculate_county_stats()` function, which calculates:
1. The single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. The single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide
- These values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).
- If there are ties, return any of the tied counties
</div>

In [243]:
def calculate_county_stats(covid_data):
    
    # YOUR CODE HERE
    import math
    low_state, low_cp100k = None, math.inf
    high_state, high_cp100k = None, 0
    for state, county_details in covid_data.items():
        for county, stats in county_details.items():
            if stats['cases_per_100k'] >= high_cp100k:
                high_state = f"{county} ({state})"
                high_cp100k = stats['cases_per_100k']
            if stats['cases_per_100k'] <= low_cp100k:
                low_state = f"{county} ({state})"
                low_cp100k = stats['cases_per_100k']
            
    # END OF YOUR CODE HERE
    print(low_state + " has the lowest rate of confirmed COVID cases: " + str(low_cp100k) + " per 100k")
    print(high_state + " has the highest rate of confirmed COVID cases: " + str(high_cp100k) + " per 100k")

Run the cell below (no changes necessary) to execute your code above

In [244]:
calculate_county_stats(covid_data)

Loving County (Texas) has the lowest rate of confirmed COVID cases: 0.0 per 100k
Chattahoochee County (Georgia) has the highest rate of confirmed COVID cases: 14522.8 per 100k


<div class='exercise'><b> Exercise 2.3 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths()` function, which calculates:
1. The state that has the **lowest number** of deaths
2. The state that has the **highest number** of deaths

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- These values you report should be Integers, not Floating point numbers.
- If there are ties, return any of the tied counties

</div>

In [245]:
def calculate_state_deaths(covid_data):
    
    # YOUR CODE HERE
    import math
    low_state, low_deaths = None, math.inf
    high_state, high_deaths = None, 0
    for state, county_details in covid_data.items():
        state_total = sum([stats['deaths'] for _, stats in county_details.items()])
        if state_total >= high_deaths:
                high_state = state
                high_deaths = state_total
        if state_total <= low_deaths:
                low_state = state
                low_deaths = state_total
        
    # END OF YOUR CODE HERE

    print(low_state + " has the fewest COVID deaths: " + str(low_deaths))
    print(high_state + " has the most COVID deaths: " + str(high_deaths))

Run the cell below (no changes necessary) to execute your code above

In [246]:
calculate_state_deaths(covid_data)

Wyoming has the fewest COVID deaths: 42
New York has the most COVID deaths: 32610


<div class='exercise'><b> Exercise 2.4 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate()` function, which calculates:
1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- To calculate a state's population, we are asserting that is sufficient to sum the population over all counties, and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- **If a county has reported 0 COVID cases,** then we should ignore this county as we estimate its county population. Thus, that county would contribute 0 to its state population total.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [247]:
def calculate_state_deathrate(covid_data):
    
    # YOUR CODE HERE
    import math
    low_state, low_deathrate = None, 0
    high_state, high_deathrate = None, math.inf
    for state, county_details in covid_data.items():
        county_pop = [(stats['cases'] / stats['cases_per_100k']) * 100000 \
                      for _, stats in county_details.items() if stats['cases']]
        state_pop = sum(county_pop)
        state_deaths = sum([stats['deaths'] for _, stats in county_details.items()])
        state_dr = math.floor(state_pop / state_deaths)
#         print(f"{state} has {state_pop} population and {state_deaths} deaths for a rate of {state_dr}")
        if state_dr <= high_deathrate:
                high_state = state
                high_deathrate = state_dr
        if state_dr >= low_deathrate:
                low_state = state
                low_deathrate = state_dr
        
    # END OF YOUR CODE HERE
    print(low_state + " has the lowest COVID death rate; 1 out of every " + str(low_deathrate) + " people has died")
    print(high_state + " has the highest COVID death rate; 1 out of every " + str(high_deathrate) + " people has died")

Run the cell below (no changes necessary) to execute your code above

In [248]:
calculate_state_deathrate(covid_data)

Alaska has the lowest COVID death rate; 1 out of every 16566 people has died
New Jersey has the highest COVID death rate; 1 out of every 554 people has died


## 3. PANDAS (36 pts)
What if we wanted to observe more than just the single-most extreme counties and states? What if we wanted to inspect all states, after having sorted the data by some feature? As you saw in the above exercises, doing the most basic analytics is possible, but it can quickly become cumbersome. As we learned in class, PANDAS is a great library that provides data structures that are highly useful for data analysis.

<div class='exercise'><b> Exercise 3.1 [10 pts]: Converting to PANDAS</b>

In Exercise 2, we worked with `covid_data`, which is comprises of some combination of lists and/or dictionaries.

Complete the `convert_to_pandas()` function, which converts `covid_data` to a PANDAS DataFrame, whereby:
- Each row corresponds to a unique county
- The 5 columns are:
    - county
    - state
    - \# total covid cases (Integer)
    - \# covid cases per 100k (Integer)
    - \# covid deaths (Integer)
- The columns should be titled **exactly** as listed above

**NOTE:**
- If there exists multiple counties with the same name, each of which belonging to a different state, then there should be a distinct row for each.
- The 3 columns that correspond to COVID counts should all be Integers (e.g., 1498), not Floating point digits (e.g., 1498.0)
</div>

In [249]:
def convert_to_pandas(covid_data):
    
    # YOUR CODE HERE
    cols = ['county', 'state', '# total covid cases', '# covid cases per 100k', '# covid deaths']
    data = []
    for state, county_details in covid_data.items():
        for county, stats in county_details.items():
            l = [county,state,stats['cases'],stats['cases_per_100k'],stats['deaths']]
            data.append(l)
    covid_df = pd.DataFrame(data, columns=cols)
    covid_df.head()
    # END OF YOUR CODE HERE
    
    return covid_df

Run the cell below (no changes necessary) to execute your code above

In [250]:
covid_df = convert_to_pandas(covid_data)
covid_df

,county,state,# total covid cases,# covid cases per 100k,# covid deaths
0,Autauga County,Alabama,1565,2801.2,24
1,Baldwin County,Alabama,4922,2204.9,47
2,Barbour County,Alabama,792,3208.3,7
3,Bibb County,Alabama,604,2697.2,9
4,Blount County,Alabama,1446,2500.6,13
...,...,...,...,...,...
3137,Sweetwater County,Wyoming,320,755.7,2
3138,Teton County,Wyoming,465,1981.8,1
3139,Uinta County,Wyoming,310,1532.7,2
3140,Washakie County,Wyoming,111,1422.2,6


<div class='exercise'><b> Exercise 3.2 [5 pts]: Simple analytics</b>

Complete the `calculate_county_stats2()` function, **which should obtain identical information as problem 2.2, but now using the PANDAS `covid_df` DataFrame.**

That is, it should calculates:
1. the single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. the single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).

</div>

In [251]:
def calculate_county_stats2(covid_df):

    # YOUR CODE HERE
    cols = ['county', 'state', '# total covid cases', '# covid cases per 100k', '# covid deaths']
    low = covid_df.iloc[covid_df[cols[3]].idxmin()]
    high = covid_df.iloc[covid_df[cols[3]].idxmax()]
    # END OF YOUR CODE HERE
    print(low[cols[0]] + " (" + low[cols[1]] + ") has the lowest rate of confirmed COVID cases: " + str(low[cols[3]]) + " per 100k")
    print(high[cols[0]] + " (" + high[cols[1]] + ") has the highest rate of confirmed COVID cases: " + str(high[cols[3]]) + " per 100k")

Run the cell below (no changes necessary) to execute your code above

In [252]:
calculate_county_stats2(covid_df)

Bristol Bay Borough (Alaska) has the lowest rate of confirmed COVID cases: 0.0 per 100k
Chattahoochee County (Georgia) has the highest rate of confirmed COVID cases: 14522.8 per 100k


<div class='exercise'><b> Exercise 3.3 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths2()` function, **which should obtain identical information as problem 2.3, but now using the PANDAS `covid_df` DataFrame.**
1. the state that has the **lowest number** of deaths
2. the state that has the **highest number** of deaths

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Integers, not Floating point numbers.
</div>

In [253]:
def calculate_state_deaths2(covid_df):
    
    # YOUR CODE HERE
    state_deaths = covid_df.groupby('state')['# covid deaths'].sum()
    # END OF YOUR CODE HERE
    print(state_deaths.idxmin() + " has the fewest COVID deaths: " + str(state_deaths.min()))
    print(state_deaths.idxmax() + " has the most COVID deaths: " + str(state_deaths.max()))

Run the cell below (no changes necessary) to execute your code above

In [254]:
calculate_state_deaths2(covid_df)

Wyoming has the fewest COVID deaths: 42
New York has the most COVID deaths: 32610


<div class='exercise'><b> Exercise 3.4 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate2()` function, **which should obtain identical information as problem 2.4, but now using the PANDAS `covid_df` DataFrame.**

1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- Just as in, 2.4, to calculate a state's population, we are asserting that is sufficient to sum the population over all counties -- and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- Just as in 2.4, counties with 0 COVID cases should contibute 0 to the total population of the state.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [283]:
def calculate_state_deathrate2(covid_df):
    
    # YOUR CODE HERE
    cols = ['county', 'state', '# total covid cases', '# covid cases per 100k', '# covid deaths']
    import math
    def calc_population(row):
        if row['# total covid cases'] == 0:
            return 0
        return math.floor(row['# total covid cases'] / row['# covid cases per 100k'] * 100000)
    
    def calc_mortality(row):
        return math.floor(row['population'].sum() / row['# covid deaths'].sum())
    
    covid_df['population'] = covid_df.apply(lambda x: calc_population(x), axis=1)
    state_dr = covid_df.groupby('state').apply(lambda x: calc_mortality(x))
    
    # END OF YOUR CODE HERE
    print(state_dr.idxmax() + " has the lowest COVID death rate; 1 out of every " + str(state_dr.max()) + " people has died")
    print(state_dr.idxmin() + " has the highest COVID death rate; 1 out of every " + str(state_dr.min()) + " people has died")

Run the cell below (no changes necessary) to execute your code above

In [284]:
calculate_state_deathrate2(covid_df)

state
Alabama        2086
Alaska        16566
Arizona        1369
Arkansas       3166
California     2756
dtype: int64
Alaska has the lowest COVID death rate; 1 out of every 16566 people has died
New Jersey has the highest COVID death rate; 1 out of every 554 people has died


These are highly alarming and tragic statistics, and doing calculations like this can really put the severity of the virus into a grounded perspective. In order to perfectly understand the virus and its spread, everyone would be tested and we would have contact tracing. Without getting into socio-political issues, our point is that (1) we wish to better understand the virus' effects; (2) naturally, any real-world data is messy, and thus we will never have _perfect_ data.


Let's now attempt to understand _some_ of the uncertainty around our COVID data. It's reasonable to believe that the # of COVID deaths is fairly reliable. That is, there are inevitably some false negatives -- people who died of COVID but were not accounted for, as other conditions were listed as the cause. However, the number of false positives is probably minimal -- if someone was denoted as dying from COVID, it's probably true. It's also the case that every disease has a mortality rate. For example, if 1,000 randomly-selected people contracted COVID, $N\%$ of them will die. We'd imagine that this percentage should be pretty constant throughout all people in the United States. Of course, we can think of reasons for this rate to not be perfectly consistent, as some people are at higher risk (e.g., older folks, people with pre-existing conditions, etc). Yet, we can imagine that this natural *variance* in the population to be fairly uniform throughout the USA at large. To this end, if all counties were equal in their **testing**, we ought to see a consistent ratio between: (a) the # of people who died from COVID; and (b) the # of people who tested positive for COVID. Within the medical domain, this ratio is referred to as the `case_fatality_rate`. For example, if 750 people tested positive for COVID, and 75 of those people died, then our `case_fatality_rate` would be 0.1 (meaning 10%).

<div class='exercise'><b>Exercise 3.5 [5 pts]: Further analytics</b>
    
Complete the `add_death_stats()` function below, which should add 2 new columns:
- `case_fatality_rate` and
- `# covid deaths per 100k`

And return the updated DataFrame **sorted by case_fatality_rate in ascending order** 

**NOTES:**

- `add_death_stats()` should return a new DataFrame that has 8 columns:
    - county
    - state
    - population
    - \# total covid cases
    - \# covid cases per 100k
    - \# total covid deaths
    - \# covid deaths per 100k
    - case_fatality_rate
- DataFrame should be sorted by `case_fatality_rate` in ascending order
- Again, the values for `case_fatality_rate` should be < 1. A value of 1 would mean that 100% of people who tested positive for COVID also died.
- `# covid deaths per 100k` is simply defined as the # of COVID deaths for every 100,000 people. We calculate this on a per-county basis.
</div>

In [305]:
def add_death_stats(covid_df):
    
    # YOUR CODE HERE
    covid_df.head()
    covid_df['# covid deaths per 100k'] = covid_df['# covid deaths'] / covid_df['population'] * 100000
    covid_df['case_fatality_rate'] = covid_df['# covid deaths'] / covid_df['# total covid cases']
    covid_df = covid_df.sort_values('case_fatality_rate', ascending=False)
    print(covid_df.head(25))
    # END OF YOUR CODE HERE
    return covid_df

Run the cell below (no changes necessary) to execute your code above

In [306]:
covid_updated = add_death_stats(covid_df)
covid_updated

                   county          state  # total covid cases  \
2746  Throckmorton County          Texas                    6   
1721       Perkins County       Nebraska                   32   
1190         Waldo County          Maine                   73   
1864       Orleans County       New York                  321   
2098        Monroe County           Ohio                  106   
952         Morton County         Kansas                   13   
1222      Franklin County  Massachusetts                  436   
2653        Kenedy County          Texas                    7   
87     Petersburg Borough         Alaska                    7   
1792        Sussex County     New Jersey                 1425   
3028     Pleasants County  West Virginia                   15   
311      Middlesex County    Connecticut                 1512   
1794        Warren County     New Jersey                 1418   
1878       Steuben County       New York                  326   
2660          Knox County

,county,state,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate
2746,Throckmorton County,Texas,6,399.7,2,1501,133.244504,0.333333
1721,Perkins County,Nebraska,32,1106.9,9,2890,311.418685,0.281250
1190,Waldo County,Maine,73,183.8,14,39717,35.249389,0.191781
1864,Orleans County,New York,321,795.5,55,40351,136.303933,0.171340
2098,Monroe County,Ohio,106,776.3,18,13654,131.829501,0.169811
...,...,...,...,...,...,...,...,...
1751,Esmeralda County,Nevada,0,0.0,0,0,NaN,NaN
2242,Wheeler County,Oregon,0,0.0,0,0,NaN,NaN
2539,Borden County,Texas,0,0.0,0,0,NaN,NaN
2657,King County,Texas,0,0.0,0,0,NaN,NaN


<div class='exercise'><b> Exercise 3.6 [6 pts]: Reflection:</b> Data Analysis allows us to better understand a system or scenario.
</div>

<div class='exercise'><b>Exercise 3.6: Q1 (2  of 6 pts)</b>
    
Having looked at the results from Exercises 3.3, 3.4, and 3.5, what are some trends you've noticed and any conclusions you have? (2-3 sentences)?</div>

YOUR RESPONSE HERE

Trivially, I understand the concept of "x per 100k" much better, and it drives some of the more grave insights. In particular, it appears that smaller communities with outbreaks can have statistics that may be "outliers". This makes sense, as the law of large numbers means certain counties will have large samples and drive towards the expected mean case fatality rate. It is difficult to reach conclusions without contextualizing these results, and attempting to directly answer a question. The case fatality rates stand out as quite high, and bring to mind questions about obtaining test administration data.

<div class='exercise'><b>Exercise 3.6: Q2 (2 of 6 pts)</b>
    
Having looked at the results from Exercise 3.5 (i.e., `covid_updated` DataFrame), do you think the original data is reliable and accurate? Are there any potential biases that you're aware of or concerned about? Please explain (3-5 sentences).</div>

YOUR RESPONSE HERE

The data appears to be reliable, in that the facts and figures are probably not in question, outside of normal (i.e. rare) measurement error. The data collection may need to be contextualized based on circumstance, such as the test availability. If the test availability is low, and only the very sick are tested, the overall case mortality rate may be slightly inflated above an actual population case mortality. The potential bias introduced by considering counties with zero cases to have zero population could be impactful, if in fact the population of those counties was large, given the recorded transmission rate. From a spot check of the tail of our dataframe, the counties do indeed have small populations, and therefore the exposure to invalid conclusions from this source of bias is small.

<div class='exercise'><b>Exercise 3.6: Q3 (1 of 6 pts)</b>
    
If a county has 15 confirmed deaths, how many cases would you expect? What would you expect its population to be? Explain why (1-2 sentences in total)?

**NOTE:** For this question, we aren't evaluating the accuracy of your answer but your thought-process and reasoning.
</div>

YOUR RESPONSE HERE

Given a case fatality of ~10%, I would expect ~150 cases. Similarly, given a deaths per 100k of ~100, I would expect the population to be ~15,000.

<div class='exercise'><b>Exercise 3.6: Q4 (1 of 6 pts)</b>
    
What further questions do you wish to answer about COVID, including ones that may not be possible to answer from this data alone (e.g., Is there a correlation between the average age of people in a county and the # of COVID deaths)? Write at least 3 of your questions.</div>

YOUR RESPONSE HERE

- Is there a correlation between county population density (population / square meter) and # total cases?
- Is there a correlation between county median income and case fatality rate?
- Is there a correlation between test administration frequency (% population coverage) and case fatality rate?

## 4. MORE DATA (25 pts)
In order to better understand how COVID (and the testing thereof) has impacted our world, we could look at how it relates to demographics, income, education, health, and political voting. For this exercise, we will make use of `election2016_by_county.csv`.

<div class='exercise'><b>Exercise 4.1 [4 pts]: Load more data</b>

Complete the `merge_data()` function, which should:
1. First, load `election2016_by_county.csv` as a new DataFrame.
2. Then, using the state and county names (case-sensitive) in both DataFrames, merge this new DataFrame with your existing `covid_updated`.
3. Return the merged DataFrame

The returned `merged` DataFrame should contain all 7 columns from `covid_updated`:
- county
- state
- \# total covid cases
- \# covid cases per 100k
- \# covid deaths
- population
- case_fatality_rate

along with these 14 columns from `election2016_by_county.csv`:
- hispanic
- minority
- female
- unemployed
- income
- nodegree
- bachelor
- inactivity
- obesity
- density
- cancer
- votergap
- trump
- clinton

**NOTES:**
- We are dropping two columns from `election2016_by_county.csv`:
    - fipscode
    - population
- Do not attempt to manually fix any of the state or county names. That is, **our merging should require the state and county names to be identical (case-sensitive) between the two DataFrames.** If there is a discrepancy between the two, do not worry about adjusting these names to find a perfect match.

**HINT:** there are many ways to solve this, but you may find the [pandas.merge()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) function can be really helpful
</div>

In [316]:
def merge_data(covid_updated, filepath):
    
    # YOUR CODE HERE
    cols = ['state', 'county', 'hispanic', 'minority', 'female', 'unemployed', 'income', 'nodegree', \
           'bachelor', 'inactivity', 'obesity', 'density', 'cancer', 'votergap', 'trump', 'clinton']
    df = pd.read_csv(filepath, usecols=cols)
    merged = covid_updated.merge(df, on=['state', 'county'])
    return merged
    # END OF YOUR CODE HERE
    #return ____

Run the cell below (no changes necessary) to execute your code above

In [317]:
merged = merge_data(covid_updated, 'election2016_by_county.csv')

As mentioned above, the merging requires exact matching between the two DataFrames' `state` and `county` columns. Thus, some mismatches will occur, yielding our `merged` DataFrame to have fewer rows than `covid_updated` and `election2016_by_county.csv`.

<div class='exercise'><b>Exercise 4.2 [5 pts]: Data Construction / Understanding</b>
</div>

<div class='exercise'><b>Exercise 4.2: Q1 (1 of 5 pts)</b>
    
Compared to `covid_updated`, how many rows were lost during this merging process to create `merged`? Running the cell below should print to the screen your answer.
</div>

In [319]:
# YOUR CODE HERE
print(f"{len(covid_updated.index) - len(merged.index)} rows were lost.")
# END OF YOUR CODE HERE

63 rows were lost.


<div class='exercise'><b>Exercise 4.2: Q2 (2 of 5 pts)</b>  

List the county and state of *at least 3* such rows that exist in `covid_updated` but didn't make it into `merged`. Running the cell below should print to the screen your answer.
</div>

In [328]:
# YOUR CODE HERE
diff = covid_updated.merge(merged, how='outer', indicator=True).loc[lambda x: x._merge == 'left_only']
print(diff[['state', 'county']].head(3))
# YOUR CODE HERE

       state                     county
8     Alaska         Petersburg Borough
27  Virginia      Falls Church, City of
46  Virginia  Colonial Heights, City of


<div class='exercise'><b>Exercise 4.2: Q3 (2 of 5 pts)</b>  
? If we needed to be highly thorough and needed comprehensive data coverage, do you have any suggestions on how we could quickly, soundly fix most or all of them? (Write 2-3 sentences.)
    
<b>NOTE: Please do not actually fix these mismatches; for this Exercise, it's okay that the `merged` DataFrame is smaller than `covid_updated`</b>
</div>

*Your answer here*

I believe that the municipality type (city, village, People's Republic) can be removed from the original COVID dataset, and that the counties would match more often.

This past example demonstrates how easy it is for data to become messy. It also shows the importance of paying close attention to your data in order to understand what you are working with.

Our `case_fatality_rate` column can be viewed as an approximation of how effective and thorough *COVID testing* is for a given county.

Our `# covid deaths` column can be viewed as an extreme indication of how severe *COVID* has impacted a given county.

Our `# covid cases per 100k` column be viewed as middle-ground between the two aforementioned features. That is, it measures the impact of the disease and is influenced by the thoroughness of COVID testing.

Using these three informative features, we can inspect how impacted each county is, while correlating this with other features of each county, such as income-level, health metrics, demographics, etc. 

<div class='exercise'><b>Exercise 4.3 [2 pts]: Cleaning the data</b>

Before we do any further analysis, we first notice that some counties haven't encountered a single COVID death (usually ones with very small populations), thus providing us with little information. Write code in the cell below to update the `merged` DataFrame so that all rows with 0 deaths are removed.

In [330]:
# YOUR CODE HERE
merged = merged.loc[merged['# covid deaths'] > 0]
# END OF YOUR CODE HERE

,county,state,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,...,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton
0,Throckmorton County,Texas,6,399.7,2,1501,133.244504,0.333333,12.793,3.800,...,43618,14.5,20.7,25.3,26.8,14.2,185.6,78.094,88.490,10.396
1,Perkins County,Nebraska,32,1106.9,9,2890,311.418685,0.281250,3.940,1.868,...,56080,8.2,20.7,26.2,24.5,6.4,349.9,73.484,84.767,11.283
2,Waldo County,Maine,73,183.8,14,39717,35.249389,0.191781,1.320,3.246,...,46598,8.1,29.0,21.0,28.3,13.3,263.3,-0.283,45.859,46.142
3,Orleans County,New York,321,795.5,55,40351,136.303933,0.171340,4.829,9.935,...,46577,14.2,16.2,22.8,28.5,459.3,167.3,43.386,69.262,25.876
4,Monroe County,Ohio,106,776.3,18,13654,131.829501,0.169811,0.479,2.158,...,40646,14.2,10.3,35.4,36.8,252.1,220.3,46.874,71.508,24.634


Running `.describe()` allows us to quickly see summary statistics of our DataFrame

In [331]:
merged.describe()

,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton
count,2519.000000,2519.000000,2519.000000,2.519000e+03,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2496.000000,2515.000000,2515.000000,2515.000000
mean,2486.641524,1811.465383,75.441445,1.256467e+05,45.858638,0.026676,9.809944,16.264345,50.074057,5.658436,47103.411274,15.563200,20.151647,26.129853,31.191743,262.338547,227.037821,29.237205,62.372741,33.135537
std,9420.635498,1394.888414,353.411511,3.678220e+05,50.141130,0.023324,14.308817,16.264504,2.125121,1.887562,12320.852451,6.834046,9.038276,5.303283,4.560836,1858.513795,54.564978,30.129313,15.265320,14.986877
min,6.000000,59.700000,1.000000,4.030000e+02,0.910191,0.001105,0.426000,1.253000,28.479000,1.800000,21658.000000,1.900000,6.000000,8.100000,11.800000,0.100000,46.200000,-88.725000,4.122000,6.883000
25%,197.000000,867.500000,3.000000,1.634150e+04,12.960227,0.011756,2.247000,4.777500,49.656500,4.400000,38487.500000,10.400000,13.800000,22.700000,28.500000,22.000000,192.875000,12.693000,53.910500,21.806500
50%,516.000000,1478.500000,10.000000,3.491500e+04,28.534327,0.020341,4.239000,9.562000,50.489000,5.400000,45065.000000,14.300000,17.700000,26.000000,31.300000,51.700000,228.850000,35.555000,65.185000,29.791000
75%,1453.000000,2389.300000,34.000000,9.095300e+04,58.839923,0.034143,10.083000,22.698000,51.121000,6.600000,52616.000000,19.950000,24.300000,29.700000,34.100000,130.300000,262.500000,51.748500,73.691000,41.617000
max,253176.000000,14522.800000,7306.000000,1.003910e+07,484.862819,0.333333,95.824000,93.411000,56.739000,24.000000,122641.000000,53.300000,72.000000,41.400000,47.600000,69468.400000,458.300000,83.972000,90.855000,92.847000


Using the information reported from `.describe()`, we can imagine dividing our DataFrame into 4 separate bins, based on the distribution for any given feature. Specifically, based on a particular feature:
- the $1^{st}$ bin will be the data that has values between the **min** and **25%**
- the $2^{nd}$ bin will be the data that has values between **25%** and **50%**
- the $3^{rd}$ bin will be the data that has values between **50%** and **75%**
- the $4^{th}$ bin will be the data that has values between **75%** and **max**

<div class='exercise'><b>Exercise 4.4 [3 pts]: Partitioning our data</b>
    
Complete the `partition_df()` function, which takes as input:
- DataFrame to work with
- feature (e.g., obesity) to filter by
- minimum value
- maximum value

and outputs:
- a subset of the DataFrame that has values between the passed-in minimum and maximum values (inclusively) for the passed-in feature.

For example, if we called `partition_df(merged, 'obesity', 30, 45)`, it should return a subset of the `merged` DataFrame that has obesity values between 30 and 45 (and including the boundary values of 30 and 45).
</div>

In [350]:
def partition_df(df, column_name, min, max):
    # YOUR CODE HERE
    part = df[ (df[column_name] >= min) & (df[column_name] <= max) ]
    return part
    # END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 4.5: [4 pts] Exploratory Data Analysis</b>
    
Identify a few features that you're interested in, and inspect if there's any correlation with the COVID data. Specifically, simply run your `partition_df()` function below, many times, each with a different subset of the data -- select a range of values and a particular feature. For example, if I'm interested in __cancer__, I could look at the 4 quartiles (per `.describe()`) and use those ranges of values as I repeatedly execute `partition_df()`. For this exercise, after running the function several times, **write 3-5 sentences about any patterns or correlations you noticed or didn't notice but expected to find.**
</div>

In [357]:
# YOUR CODE HERE
print(f"75-100 Trump: \n{partition_df(merged, 'trump', 75, 100).describe()}")
print(f"0-25 Trump: \n{partition_df(merged, 'trump', 0, 25).describe()}\n\n==========")
print(f"25-75 Trump: \n{partition_df(merged, 'trump', 25, 75).describe()}")

# END OF YOUR CODE HERE

#partition_df(merged, 'your feature here', your_min_value, your_max_va).describe()

75-100 Trump: 
       # total covid cases  # covid cases per 100k  # covid deaths  \
count           528.000000              528.000000      528.000000   
mean            491.928030             1708.302652       10.094697   
std             616.422129             1235.723934       13.588090   
min               6.000000              107.200000        1.000000   
25%             108.000000              933.475000        2.000000   
50%             282.000000             1388.650000        5.000000   
75%             655.750000             2142.625000       13.000000   
max            4922.000000            14474.000000      118.000000   

          population  # covid deaths per 100k  case_fatality_rate    hispanic  \
count     528.000000               528.000000          528.000000  528.000000   
mean    27367.339015                40.884254            0.027426    9.133977   
std     26766.324041                38.441117            0.027867   12.384730   
min      1350.000000          

*Your answer here*

By partitioning by voting distribution, the immediate question is to understand any correlation between cases per 100k and ideological indicators. I was hypothesizing that those counties favorable to Trump may not be as diligent about social distancing and mask guidelines, and would have a higher number of cases per 100k. However, the data seems to indicate that the rate likely has more to do with demographics and geography, given the apparent relationship between density and cases per 100k statistics.

`.describe()` provides these nice summary statistics over any portion of data that we give it. Instead of iteratively inspecting several subsets of the data, let's actually split our DataFrame into new categories; instead of representing all features by floating point numbers, let's create new _categorical_ names for feature(s) based on their numbers. The code below does just this. It creates a new column, `income group` that has 4 possible values, each one corresponding to a quartile of the original `income` values. 

Run the cell below.

In [358]:
bins = [0, 38000, 45000, 52000, 200000]
names = ['income-group-1', 'income-group-2', 'income-group-3', 'income-group-4']
d = dict(enumerate(names, 1))
merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))
merged

<ipython-input-358-537dae7a419f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))


,county,state,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,...,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton,income group
0,Throckmorton County,Texas,6,399.7,2,1501,133.244504,0.333333,12.793,3.800,...,14.5,20.7,25.3,26.8,14.2,185.6,78.094,88.490,10.396,income-group-2
1,Perkins County,Nebraska,32,1106.9,9,2890,311.418685,0.281250,3.940,1.868,...,8.2,20.7,26.2,24.5,6.4,349.9,73.484,84.767,11.283,income-group-4
2,Waldo County,Maine,73,183.8,14,39717,35.249389,0.191781,1.320,3.246,...,8.1,29.0,21.0,28.3,13.3,263.3,-0.283,45.859,46.142,income-group-3
3,Orleans County,New York,321,795.5,55,40351,136.303933,0.171340,4.829,9.935,...,14.2,16.2,22.8,28.5,459.3,167.3,43.386,69.262,25.876,income-group-3
4,Monroe County,Ohio,106,776.3,18,13654,131.829501,0.169811,0.479,2.158,...,14.2,10.3,35.4,36.8,252.1,220.3,46.874,71.508,24.634,income-group-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514,Effingham County,Illinois,737,2167.1,1,34008,2.940485,0.001357,1.909,2.037,...,8.7,20.2,21.9,31.8,30.2,278.4,60.489,78.110,17.621,income-group-4
2515,Chattahoochee County,Georgia,1584,14522.8,2,10906,18.338529,0.001263,15.676,28.352,...,7.0,31.5,24.5,27.3,45.3,70.0,11.055,53.902,42.847,income-group-3
2516,La Crosse County,Wisconsin,1625,1376.9,2,118018,1.694657,0.001231,1.806,8.258,...,6.0,30.8,19.9,28.3,253.8,182.6,-9.585,42.023,51.608,income-group-3
2517,Russell County,Alabama,1678,2895.1,2,57960,3.450656,0.001192,5.285,47.987,...,18.7,15.5,30.7,36.4,82.6,216.4,-1.918,48.129,50.047,income-group-1


<div class='exercise'><b>Exercise 4.6 [5 pts]: Aggregate data</b>
    
    
Write code in the cell below to group (and display) the data according to the 4 income groups. Also, while we will still keep the same columns (i.e, features), the values of each should now represent the __average__ value of all rows that were subsumed in the making of the aggregate income-group. Your resulting DataFrame should have just 4 rows (income-group-1, income-group-2, income-group-3, income-group-4). See example in the cell below.


Since every feature (except for `# total cases`, `# covid deaths`, and `population`) was already an average value corresponding to a particular __county__, when we aggregate our data by income groups, we are effectively taking an average of an average. Many counties are being aggregated for each income-group row. This approach isn't as accurate as possible; it would be more accurate if we re-adjusted every value so that it was truly an average that was based on the total __population__ of all counties that are subsumed within a given income-group row. That's okay, though. An average of averages will suffice for the purpose of this exercise. 
</div>

In [360]:
# EXAMPLE: If our `merged` DataFrame were
# COUNTY    INCOME GROUP    BACHELOR ... (other columns, too)
#   A            2             50
#   B            1             20
#   C            1             30
#   D            2             70
#   E            3             95

# it should become
# INCOME GROUP    BACHELOR ... (other columns, too)
#   1                25
#   2                60
#   3                95

# YOUR CODE HERE
merged.groupby('income group').mean()
# END OF YOUR CODE HERE

# total covid cases                                           \
                             count         mean           std   min     25%   
income group                                                                  
income-group-1               584.0   877.785959   2796.786403   9.0  182.00   
income-group-2               670.0  1650.114925   7151.364149   6.0  164.50   
income-group-3               589.0  2489.213922   7530.411134   9.0  168.00   
income-group-4               676.0  4703.400888  14709.433958  12.0  287.25   

                                         # covid cases per 100k               \
                  50%      75%       max                  count         mean   
income group                                                                   
income-group-1  402.0   864.00   52186.0                  584.0  2584.969349   
income-group-2  430.5  1054.50  163790.0                  670.0  1703.128060   
income-group-3  488.0  1535.00   74887.0                  589.0  1605.133277   
income-group-4  970.5  3655.25  253176.0                  676.0  1430.384467   

                ...     trump         clinton                               \
                ...       75%     max   count       mean        std    min   
income group    ...                                                          
income-group-1  ...  76.30975  89.956   584.0  34.960795  18.001461  7.656   
income-group-2  ...  75.10050  88.897   670.0  29.228696  11.925420  9.838   
income-group-3  ...  72.42200  89.852   588.0  31.465724  12.573839  8.177   
income-group-4  ...  69.33500  90.855   673.0  36.900000  15.643032  6.883   

                                                   
                     25%     50%      75%     max  
income group                                       
income-group-1  20.37200  29.796  45.3385  88.727  
income-group-2  20.83875  26.388  35.1870  85.445  
income-group-3  22.29600  29.249  38.5085  80.823  
income-group-4  25.19500  34.949  46.6970  92.847  

[4 rows x 160 columns]

<div class='exercise'><b>Exercise 4.7 [2 pts]: Conclusions</b>
</div>

<div class='exercise'><b>Exercise 4.7: Q1 (1 of 2 pts)</b>
What are your conclusions/finding from this alternative view of the data? (2-4 sentences).
</div>

The alternative view has some pretty stark things that stand out. In particular, the cases per 100k being significantly higher in group-1 relative to the other groups might be some indication of the social class of so-called essential workers, a higher proportion of which are minorities. In contrast, the average case fatality rate does not vary widely between groups, indicating that the immediate clinical outcomes for the virus does not have dramatic income stratification, even if access to care is different. 

<div class='exercise'><b>Exercise 4.7: Q2 (1 of 2 pts)</b>
What are some weaknesses from this view of the data? (2-4 sentences).
</div>

Geography and the affects of cost of living don't appear to be evident in this view of the data, which may lead to invalid conclusions for certain questions, such as the effect of population density. The average of averages may offer less insight than the partitioned describe chain above into the distribution of the data. Overall, this grouped view may be better represented by a series of color-coded plots, for instance.

## Moving Forward

In this homework assignment, we've focused on gathering, parsing, and exploring data. However, what if we wanted to *predict* some behavior of the data. For example, imagine one is curious how a particular county will respond to COVID. Or, imagine we looked at counties' COVID data on a weekly basis, one could be interested in predicting the upcoming week's behavior.

Alternatively, one could be interested in *inference*, whereby we are more concerned with trying to understand __why__ and __how__ a system behaves the way it does. We might wish to understand which factors most correlate and cause a certain event to happen. This could give us insights into where certain inequalities persist.

For both *prediction* and *inference*, our computational method of solving such a task is referred to as a model. For the remainder of CS109, we will spend significant focus on various models.
</div>

## Reflection

As a reminder, this is just **one** of the homework assignments in this course, the point of which is to assess your learning and to provide both you and us with an indication as to how aligned your knowledge and skills are with our learning objectives. To this end, we encourage you to reflect on your progress, strengths, and weaknesses and to make changes, if necessary, to accomplish your goals. Likewise, please reach out to the TFs and teaching staff if you need help. We want everyone to feel comfortable in being honest about these elements, with both herself/himself and us. For these purposes, we will ask you several times throughout the semester to complete an anonymous poll.